In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
from nltk import bigrams, trigrams, word_tokenize
from collections import Counter, defaultdict
from collections import OrderedDict
from math import log2
from time import time
import pickle
import operator
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip "drive/MyDrive/Quora-QnA/quora-question-pairs" -d "quora-question-pairs"
!unzip "quora-question-pairs/train.csv.zip"
!unzip "quora-question-pairs/test.csv.zip"

Archive:  drive/MyDrive/Quora-QnA/quora-question-pairs.zip
  inflating: quora-question-pairs/sample_submission.csv.zip  
  inflating: quora-question-pairs/test.csv  
  inflating: quora-question-pairs/test.csv.zip  
  inflating: quora-question-pairs/train.csv.zip  
Archive:  quora-question-pairs/train.csv.zip
  inflating: train.csv               
Archive:  quora-question-pairs/test.csv.zip
  inflating: test.csv                


In [4]:
df = pd.read_csv("train.csv", index_col = 'id')
df.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
df[df['is_duplicate'] == 0].head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [8]:
df[df['is_duplicate'] == 1].head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [ ]:
df["question1"] = df["question1"].str.lower()
df["question2"] = df["question2"].str.lower()

In [ ]:
df = df.drop(df.loc[df['question1'].str.len() <= 5].index)

In [ ]:
indexes_to_delete= []
for index in range(len(df)):
  row = df.iloc[index]
  if type(row['question1']) != type("")  or type(row['question2']) != type(""):
    indexes_to_delete.append(index)
  
for j in reversed(indexes_to_delete):
  df = df.drop(j)

len(indexes_to_delete)

3

In [ ]:
# vocab = defaultdict(lambda : 0)
# vocab_size = 0
# for i in df['question1']:
#   for word in nltk.word_tokenize(i):
#     if(vocab[vocab_size] == 0):
#       vocab_size+=1
# for i in df['question1']:
#   for word in i:
#     if(vocab[vocab_size] == 0):
#       vocab_size+=1

In [ ]:
def calculate_p_model(dataset, V, vocab_size):
  p_model = defaultdict(lambda : 0)
  q_model = defaultdict(lambda : 0)
  errcount= 0

  total_count = 0
  for index, row in dataset.iterrows():
    try:
      sentence1 = nltk.word_tokenize(row['question1'])
      for word in sentence1:
        if(V[word] == 0):
          vocab_size+=1
          V[word] = 1
        if row['is_duplicate'] == 0 and word in nltk.word_tokenize(row['question2']):
            p_model[word] += 1 
        elif row['is_duplicate'] == 1 and word in nltk.word_tokenize(row['question2']):
            q_model[word] += 1
    except:
      errcount+=1


  for index, row in dataset.iterrows():
    try:
      sentence2 = nltk.word_tokenize(row['question2'])
      for word in sentence2:
        if(V[word] == 0):
          vocab_size+=1
          V[word] = 1
    except:
      errcount+=1
    
  print("errcount: ", errcount)
  return p_model, q_model, V, vocab_size

In [ ]:
def normalize_pq_models(dataset, p_model, q_model, vocab_size):
  d_count = defaultdict(lambda : 0)
  errcount= 0

  for index, row in dataset.iterrows():
    try:
      sentence2 = nltk.word_tokenize(row['question2'])
      for word in sentence2:
        d_count[word]+=1
    except:
      errcount+=1

  for word in p_model:
    try:
      p_model[word] = p_model[word]/(d_count[word])
    except:
      errcount+=1
  for word in q_model:
    try:
      q_model[word] = q_model[word]/(d_count[word])
    except:
      errcount+=1

  print("errcount: ", errcount)
  return p_model, q_model

###### Testing the above functions

In [ ]:
# question1 = ["hi how do you do", "how is photosynthesis defined ?", "i have been to London", "this is a model"]
# question2 = ["hi mate what is this ?", "what is photosynthesis ?", "i have Not been to london", "hi hello world"]
# is_duplicate = [1,1,0,0]
# test = pd.DataFrame(np.array([question1, question2, is_duplicate]).transpose(), columns = ['question1', 'question2', 'is_duplicate'])
#print(test)
# vocab_size = 0
# V = defaultdict(lambda : 0)
# p_model, q_model, V, vocab_size = calculate_p_model(test,V, vocab_size)
# print(q_model)
# p_model, q_model = normalize_pq_models(test, p_model, q_model, vocab_size)
# print(p_model)

###### Applying the functions to the QQD

In [ ]:
t = time()
vocab_size = 0
V = defaultdict(lambda : 0)
p_model, q_model, V, vocab_size = calculate_p_model(df,V, vocab_size)
print("time taken : ",round((time() - t)/60,2), "m ", round((time() - t)%60,2), "s ")

errcount:  5
time taken :  13.91 m  54.87 s 


In [ ]:
p_model['not']

1033

In [ ]:
q_model['not']

851

In [ ]:
t = time()
p_model, q_model = normalize_pq_models(df, p_model, q_model, vocab_size)
print("time taken : ",round((time() - t)/60,2), "m ", round((time() - t)%60,2), "s ")

errcount:  2
time taken :  1.25 m  15.19 s 


In [ ]:
p_model['not']

0.09516351911561492

In [ ]:
q_model['not']

0.07839705204974666

###### Calculating KL Divergence based on q and p

The Kullback-Leibler Divergence score, or KL divergence score, quantifies how much one probability distribution differs from another probability distribution.

KL divergence can be calculated as the negative sum of probability of each event in P multiplied by the log of the probability of the event in Q over the probability of the event in P.

KL(P || Q) = – sum x in X P(x) * log(Q(x) / P(x))
The value within the sum is the divergence for a given event.

In [ ]:
def KLD(p, q):
	return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))
 

def calculate_kl_divergences(p_model, q_model):
  kl_div = defaultdict(lambda : 0)

  for word in p_model.keys():
    if word in q_model:

      P_probability = round(p_model[word],2)
      Q_probability = round(q_model[word],2)

      if(P_probability > 0 and P_probability < 1 and Q_probability > 0 and Q_probability < 1 ):
        p = list([P_probability, 1 - P_probability])
        q = list([Q_probability, 1 - Q_probability])
        kl_div[word] = KLD(p, q)
  
  return kl_div

In [ ]:
kl_div = calculate_kl_divergences(p_model, q_model)

In [ ]:
[0.4,0.6]
[0.8,0.2]

0.003654835734131903
0.02985104330111349


In [ ]:
a_file = open("kl_div.pkl", "wb")
pickle.dump(dict(kl_div), a_file)
a_file.close()

In [ ]:
a_file = open("p_model.pkl", "wb")
pickle.dump(dict(p_model), a_file)
a_file.close()

In [ ]:
a_file = open("q_model.pkl", "wb")
pickle.dump(dict(q_model), a_file)
a_file.close()

In [ ]:
kl_div['never']

0.0005460756963779761

In [ ]:
print(kl_div['not'])


0.003654835734131903


In [ ]:
print(kl_div['studies'])

0.02985104330111349


In [ ]:
print(kl_div['medicine'])

0.007051633224687742


In [ ]:
print(kl_div['physics'])

0.05029203449171893
